In [15]:
import csv
import pandas as pd
import numpy as np
from sklearn.cluster import MeanShift

In [2]:
#1 Obtaining the data

data
with open('checkins.dat') as input_file:        
    newLines = []
    for line in input_file:
        newLine = [x.strip() for x in line.split('|')]
        if len(newLine) == 6 and newLine[3] and newLine[4]:
            newLines.append(newLine)

with open('checkins.csv', 'w') as output_file:
    file_writer = csv.writer(output_file)
    file_writer.writerows(newLines)

In [3]:
data = pd.read_csv('checkins.csv')
print len(data)
data.head()

396634


,id,user_id,venue_id,latitude,longitude,created_at
0,984222,15824,5222,38.895112,-77.036366,2012-04-21 17:43:47
1,984234,44652,5222,33.800745,-84.410520,2012-04-21 17:43:43
2,984291,105054,5222,45.523452,-122.676207,2012-04-21 17:39:22
3,984318,2146539,5222,40.764462,-111.904565,2012-04-21 17:35:46
4,984232,93870,380645,33.448377,-112.074037,2012-04-21 17:38:18


In [4]:
# Extracting coords
coords = data[['latitude','longitude']].values
coords

array([[  38.8951118,  -77.0363658],
       [  33.800745 ,  -84.41052  ],
       [  45.5234515, -122.6762071],
       ...,
       [  42.7653662,  -71.467566 ],
       [  42.439479 ,  -83.7438303],
       [  42.331427 ,  -83.0457538]])

In [6]:
%%time
# Clustering
clusteriser = MeanShift(bandwidth=0.1, n_jobs=8)
labels = clusteriser.fit_predict(coords)

CPU times: user 21min 43s, sys: 12.7 s, total: 21min 56s
Wall time: 21min 54s


In [26]:
# Selecting clusters with more than 15 elements
n_clusters = max(labels)
suit_labels = np.array([i for i in range(n_clusters) if np.count_nonzero(labels == i) > 15])
n_suit_clusters = len(suit_labels)
print labels
print suit_labels
print n_clusters
print n_suit_clusters

[   4    7   29 ...  364 1064   96]
[   0    1    2 ... 3725 4039 5192]
5535
1304


In [35]:
# Calculating cluster centers
cl_centers = np.array([[np.array([coords[j][0] for j in range(len(labels)) if labels[j] == i]).mean(), 
                        np.array([coords[k][1] for k in range(len(labels)) if labels[k] == i]).mean()] 
                       for i in suit_labels])
print cl_centers

[[  40.71755421  -73.98945146]
 [  33.44827632 -112.07391798]
 [  41.87818756  -87.62983757]
 ...
 [  42.15607977  -88.4376275 ]
 [  14.40962866  121.01180815]
 [  40.21042943  -77.00956502]]


In [39]:
# Calculating minimal distances
offices = np.array([[33.751277, -118.188740],
           [25.867736, -80.324116],
           [51.503016, -0.075479],
           [52.378894, 4.885084],
           [39.366487, 117.036146],
           [-33.868457, 151.205134]])
dist = np.array([min([np.linalg.norm(cl_centers[i] - offices[j], ord=2)
                      for j in range(6)]) for i in range(n_suit_clusters)])
dist

array([16.14450605,  6.12232453, 17.59852626, ..., 18.19723087,
       25.27153869, 14.72070321])

In [41]:
# Obtaining the answer
cl_centers[np.argmin(dist)]

array([-33.86524223, 151.20746318])